In [1]:
#imports
import os
os.chdir('../') 

In [26]:
from lib.readfile import ReadFile as rf
from lib.handle_null_values import handle_null_values as null
from lib.datetime_formatting import DatetimeFormatting as dfmt
from lib.split_time_variable import split_time_variable as split_t
from lib.correct_variable_types import correct_variable_types as vartype
from lib.handle_null_values import handle_null_values as nulval

"""Read the file"""

r= rf()
train= r.read(address="lib/data/2009.csv")

In [4]:
from lib.readfile import ReadFile as rf
from lib.handle_null_values import handle_null_values as null
from lib.datetime_formatting import DatetimeFormatting as dfmt
from lib.split_time_variable import split_time_variable as split_t
from lib.correct_variable_types import correct_variable_types as vartype
from lib.handle_null_values import handle_null_values as nulval
import pandas as pd
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator
from pyspark import SparkContext
from pyspark.sql import SparkSession

/Users/muddassarsharif/Desktop/muddassar/production/API/env4/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [60]:
# imports
from pyspark import SparkContext
from pyspark.sql import SparkSession

spark_context = SparkContext.getOrCreate()
spark = SparkSession(spark_context)

In [61]:
spark

In [7]:

test = spark.read.parquet("lib/data/king_train.parquet")

In [8]:
train = spark.read.parquet("lib/data/king_test.parquet")

In [9]:
train_y= train.select("sold_price").toPandas()
train= train.drop("sold_price")

In [10]:
test_y= test.select("sold_price").toPandas()
test= test.drop("sold_price")

In [54]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.sql import SQLContext as spark
from pyspark.sql.functions import isnan, when, count, col, from_unixtime
from pyspark.ml import Pipeline, Transformer
from pyspark.ml.feature import Imputer
from pyspark.sql.functions import year, month, dayofmonth
from pyspark.sql.functions import *
from pyspark.sql.types import DoubleType, TimestampType
from pyspark.ml.util import DefaultParamsReadable, DefaultParamsWritable, JavaMLReadable, JavaMLWritable
from datetime import datetime
from pyspark.sql.functions import udf
from pyspark.sql.types import DateType
from pyspark.ml.param.shared import *
import random
import string
import pyspark
from pyspark.sql import SparkSession
import time



class transform_pipeline():
    def __init__(self):
        #variables
        self.pipeline=None
        self.stages=[]
        # selected_columns specifies the order of the columns
        self.param={'variables_updated':False, 'columns': [], 'y_variable': None, 'time_variable':[], 'selected_columns':[],  'correct_var_types':{}}

        self.sc=pyspark.SparkContext.getOrCreate()

        self.sql=pyspark.sql.SparkSession(self.sc)

    #methods
    def set_parameter(self, key, value):
        self.param[key]= value

    def get_parameter(self, key=None):
        if key==None:
            return self.param
        return self.param[key]


    #methods
    def save_pipeline(self, bucket= 'mltrons', path=None):
        if path==None:
            #generate own path
            path= ''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(20))

        self.pipeline.save(path)
        return path

    def load_pipeline(self, df= None, bucket= 'mltrons', path=None):
        if df== None:
            print("PLease provide test df")
            return False
        # pipeline = Pipeline.load('s3n://'+bucket+'/'+ path)
        pipeline = Pipeline.load(os.path.join(settings.BASE_DIR, 'spark', path))
        self.pipeline= pipeline.fit(df)
        return self.pipeline

    def remove_y_from_variables(self, variables, y_var, training=True):
        new_variable = []
        if training == True:
            for v in variables:
                if y_var == v[0]:
                    pass
                else:
                    new_variable.append(v)
        return new_variable

    def convert_y_to_float(self, df, y_var):
        return df.withColumn(y_var, col(y_var).cast(DoubleType()))

    def transform(self, df=None):
        if df==None:
            print("Please provide dataframe to build")
            return False

        if self.pipeline==None:
            print("Please build or load the pipeline first.")
            return False
        df= self.pipeline.transform(df)
        return df

    def build_pipeline(self, df=None):
        # make sure all the variables are available
        if self.param['variables_updated']==False:
            print("Summary parameters are missing.")
            return False
        if df == None:
            print("Please provide dataframe to build a pipeline")
            return False


        # convert int to double
#         self.categorical_to_float()
        self.int_to_double()

        self.handle_missing_values()
        
        
#         try:
#             self.int_to_double()
#         except Exception as e:
#             print(e, "int to double")
#             return False

#         # categories: string to integer
#         try:
#             self.categorical_to_float()
#         except Exception as e:
#             print(e, "categorical to float")
#             return False

#         # handle imputations
#         try:
#             self.handle_missing_values()
#         except Exception as e:
#             print(e, "handling missing values")
#             return False


        pi= Pipeline(stages=self.stages)

        self.pipeline = pi.fit(df)

        return True


    def int_to_double(self):
        co= ['bigint', 'int', 'double', 'float']
        for colum in self.param["columns"]:
            if colum[1] in co:
                # time to transform
                change = change_type(column=colum[0])
                self.stages+= [change]


    def categorical_to_float(self):

        for column in self.param["columns"]:
            if self.param['time_variable']!=[]:
                if column[0]== self.param['time_variable'][0]:
                    continue
            if column[1]=='string':
                # time to transform
                stringIndexer = StringIndexer(inputCol=column[0], outputCol=column[0] + "Index").setHandleInvalid("keep")
                self.param["selected_columns"].append(column[0] + "Index")
                self.stages+= [stringIndexer]
                d = drop(column[0])
                self.stages+= [d]
            else:
                self.param["selected_columns"].append(column[0])

    def handle_missing_values(self):

        imputer = Imputer(
                            inputCols=self.param["selected_columns"],
                            outputCols=self.param["selected_columns"]
                        )
        self.stages+= [imputer]




class change_type(Transformer,DefaultParamsReadable, DefaultParamsWritable):

    column = Param(Params._dummy(), "column", "column for transformation", typeConverter=TypeConverters.toString)

    def __init__(self, column=''):
        super(change_type, self).__init__()

        self._setDefault(column= column)
        self.setColumn(column)

    def getColumn(self):
        """
        Gets the value of withMean or its default value.
        """
        return self.getOrDefault(self.column)


    def setColumn(self, value):
        """
        Sets the value of :py:attr:`withStd`.
        """
        return self._set(column=value)


    def _transform(self,df):
        return df.withColumn(self.getColumn(), col(self.getColumn()).cast(DoubleType()))


class drop(Transformer,DefaultParamsReadable, DefaultParamsWritable):

    column = Param(Params._dummy(), "column", "column for transformation", typeConverter=TypeConverters.toString)

    def __init__(self, column=''):
        super(drop, self).__init__()

        self._setDefault(column= column)
        self.setColumn(column)

    def getColumn(self):
        """
        Gets the value of withMean or its default value.
        """
        return self.getOrDefault(self.column)


    def setColumn(self, value):
        """
        Sets the value of :py:attr:`withStd`.
        """
        return self._set(column=value)


    def _transform(self,df):
        return df.drop(self.getColumn())

In [55]:
t=transform_pipeline()
t.set_parameter('columns', train.dtypes)
t.set_parameter('y_varaible', [])
t.set_parameter('variables_updated', True)

In [56]:
ans= t.build_pipeline(train)

In [57]:
trans_train = t.transform(train)
# trans_test = t.transform(test)

In [58]:
trans_train

DataFrame[id: double, ml_number: double, mls_created_date: timestamp, mls_updated_date: string, status: string, status_before_removed: string, property_type: string, list_date: timestamp, sold_date: timestamp, list_price: double, sold_price: double, address: string, city_name: string, county: string, state_code: string, zip_code: string, area: double, latitude: double, longitude: double, bedroom_count: double, bathroom_count: double, square_foot: double, lot_size: double, occupant_name: string, occupant_phone: string, showing_information: string, public_comments: string, remarks: string, style: string, view: string, levels: string, garage: string, pool: string, heating_cooling: string, elementary_school: string, junior_high_school: string, high_school: string, interior: string, exterior: string, other_features: string, cross_street: string, unit_number: string, agent_comments: string, annual_taxes: string, apn: string, architecture: string, builder: string, building_condition: string, 

In [59]:
#run and arun tpot library
t.save_pipeline(path="lib/data/sparkpipeline130mb2")

Py4JJavaError: An error occurred while calling o69778.saveAsTextFile.
: org.apache.hadoop.mapred.FileAlreadyExistsException: Output directory file:/Users/muddassarsharif/Desktop/muddassar/production/tbDataPrep/lib/data/sparkpipeline130mb2/metadata already exists
	at org.apache.hadoop.mapred.FileOutputFormat.checkOutputSpecs(FileOutputFormat.java:131)
	at org.apache.spark.internal.io.HadoopMapRedWriteConfigUtil.assertConf(SparkHadoopWriter.scala:289)
	at org.apache.spark.internal.io.SparkHadoopWriter$.write(SparkHadoopWriter.scala:71)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply$mcV$sp(PairRDDFunctions.scala:1096)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.wi

In [118]:
from tpot import TPOTRegressor
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
import numpy as np

In [119]:
train_x= np.array(trans_train.toPandas())
test_x= np.array(trans_test.toPandas())

In [96]:
tpot = TPOTRegressor(verbosity=3)

In [ ]:
tpot.fit(train_x, np.array(train_y))

29 operators have been imported by TPOT.


/Users/muddassarsharif/anaconda3/envs/conda1/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


_pre_test decorator: _random_mutation_operator: num_test=0 Found array with 0 feature(s) (shape=(50, 0)) while a minimum of 1 is required..
_pre_test decorator: _random_mutation_operator: num_test=0 manhattan was provided as affinity. Ward can only work with euclidean distances..
_pre_test decorator: _random_mutation_operator: num_test=0 Expected n_neighbors <= n_samples,  but n_samples = 50, n_neighbors = 68.
_pre_test decorator: _random_mutation_operator: num_test=0 manhattan was provided as affinity. Ward can only work with euclidean distances..
_pre_test decorator: _random_mutation_operator: num_test=0 Found array with 0 feature(s) (shape=(50, 0)) while a minimum of 1 is required..
_pre_test decorator: _random_mutation_operator: num_test=0 Found array with 0 feature(s) (shape=(50, 0)) while a minimum of 1 is required..
_pre_test decorator: _random_mutation_operator: num_test=0 Expected n_neighbors <= n_samples,  but n_samples = 50, n_neighbors = 87.
Pipeline encountered that has pr

In [ ]:
score= tpot.score(X_test, y_test)

In [ ]:
tpot.export('lib/tpot/tpot_boston_pipeline.py')